<a href="https://colab.research.google.com/github/MiguelUrenaPliego/labTopoSAM/blob/main/segment_anything_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver

In [ ]:
import leafmap
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM

In [ ]:
m = leafmap.Map(center=[43.2294769, -7.5824514], zoom=17, height="800px") # center=[lat, lon]
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
m

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = "No has seleccionado nada"
    print("No has seleccionado nada")
    #bbox = [-122.2659, 37.8682, -122.2521, 37.8741]
bbox

In [ ]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

### Opción 2: Cargar imágenes desde archivo local
Puedes cargar tu propia imagen escribiendo la ruta. Debe ser un formato .tif con 3 canales R (rojo) G (verde) B (azul)

In [ ]:
# image = '/path/to/your/own/image.tif'

###Vemos las imágenes cargadas sobre el mapa

In [ ]:
m.layers[-1].visible = False #Si da error vuelve a ejecutar la celda de nuevo
m.add_raster(image, layer_name="Image")
m

###Opcinal: *separar imágenes grandes en partes*

In [ ]:
split_raster(image, out_dir="tiles", tile_size=(1000, 1000), overlap=0)

## Inicialización de SAM
SAM segment anything es un modelo de segmentación de imágenes desarrollado por meta https://github.com/facebookresearch/segment-anything . Tiene una versión especial para imágenes satélite https://samgeo.gishub.org/ .

Para que el procesamiento sea rápido requiere una GPU. En google collab lo hemos configurado ya en el primer paso.

Descargamos SamGeo (Puede tardar unos minutos dependiendo de tu internet)

Inicialización del modelo

In [ ]:
sam = LangSAM()

Especifica la clase que quieres segmentar

In [ ]:
text_prompt = "building roof"

## Segmentación de la imagen

El resultado se guarda en el formato GEOtiff

box_threshold [0,1] tolerancia de deteccion. Valores altos dan lugar a detecciones muy seguras pero probablemente se pasen por alto bastantes elementos. valores bajos dan lugar a detecciones menos seguras con riesgo de detectar mal algunos elementos.

text_threshold [0,1] El modelo interpreta el texto que le damos para buscar la clase necesaria. Se puede escribir en text_promt cualquier cosa. Valores altos de text_threshold dan lugar a mayor exigencia del modelo para encontrar clases parecidas al texto dado. Valores bajos daran lugar a coincidencias más generales.

In [ ]:
box_threshold = 0.24
text_threshold = 0.24

(a) Si tenemos una única imagen

In [ ]:
output_name = "mask"
sam.predict(image, text_prompt, box_threshold=box_threshold, text_threshold=text_threshold)

(b) Si tenemos las imágenes separadas en partes

In [ ]:
output_name = "merged"
sam.predict_batch(
    images='tiles',
    out_dir='masks',
    text_prompt=text_prompt,
    box_threshold=box_threshold,
    text_threshold=text_threshold,
    mask_multiplier=255,
    dtype='uint8',
    merge=True,
    verbose=True
    )

##Visualización de resultados

In [ ]:
m.add_raster('masks/{}.tif'.format(output_name), cmap='Reds', nodata=0, layer_name='Mask')
m.add_layer_manager()
m

In [ ]:
titulo = 'hola'
sam.show_anns(
    cmap='Reds',
    box_color='red',
    title=titulo,
    blend=True,
)

In [ ]:
sam.show_anns(
    cmap='Reds',
    add_boxes=False,
    alpha=0.5,
    title=titulo,
)

In [ ]:
sam.show_anns(
    cmap='Greys_r',
    add_boxes=False,
    alpha=1,
    title=titulo,
    blend=False,
    output='masks/{}.tif'.format(output_name),
)

In [ ]:
sam.raster_to_vector("masks/{}.tif".format(output_name), "masks/{}.shp".format(output_name))

In [ ]:


m.add_raster("masks/{}.tif".format(output_name), layer_name="Pools", palette="Blues", opacity=0.5, nodata=0)
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector("masks/{}.shp".format(output_name), layer_name="Vector", style=style)
m
